In [1]:
import pandas as pd

df = pd.read_csv('apt_price_2022.csv',skiprows=15, thousands= ',', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16999 entries, 0 to 16998
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   시군구       16999 non-null  object 
 1   번지        16995 non-null  object 
 2   본번        16998 non-null  float64
 3   부번        16998 non-null  float64
 4   단지명       16999 non-null  object 
 5   전용면적(㎡)   16999 non-null  float64
 6   계약년월      16999 non-null  int64  
 7   계약일       16999 non-null  int64  
 8   거래금액(만원)  16999 non-null  int64  
 9   층         16999 non-null  int64  
 10  건축년도      16997 non-null  float64
 11  도로명       16999 non-null  object 
 12  해제사유발생일   671 non-null    float64
 13  거래유형      16999 non-null  object 
 14  중개사소재지    16999 non-null  object 
dtypes: float64(5), int64(4), object(6)
memory usage: 1.9+ MB


In [2]:
# 새로운 항목 생성
df['구'] = df['시군구'].str.split(' ').str[1]
df['동'] = df['시군구'].str.split(' ').str[2]

df['평'] = round(df['전용면적(㎡)'] / 3.03,2)
df['평형'] = pd.cut(df['평'],bins = [0,10,20,30,40,50, 10000], right = False,
    labels = ['10평 미만', '10평대', '20평대', '30평대', '40평대', '50평 이상']
)

def dow(i):
    week_list = '월,화,수,목,금,토,일'.split(',')
    return week_list[i]+'요일'
계약날짜 = df['계약년월'].astype(str)+df['계약일'].astype(str)
df['계약요일'] = pd.to_datetime(계약날짜, format='%Y%m%d').dt.dayofweek
df['계약요일'] = df['계약요일'].apply(dow)

df['계약년월'] = pd.to_datetime(df['계약년월'],format='%Y%m')
df['계약년'] = df['계약년월'].dt.year
df['계약월'] = df['계약년월'].dt.month

df['거래금액(억원)'] = df['거래금액(만원)']/10000

In [3]:
# 불필요 제거
df = df.drop(['본번','부번','시군구','번지','전용면적(㎡)','거래금액(만원)'],axis=1)

In [4]:
# 결측치

df['해제사유발생일'] = df['해제사유발생일'].astype(str)
df['건축년도'] = df['건축년도'].fillna(df['건축년도'].median())
df['해제사유발생일'] = df['해제사유발생일'].fillna('NaN')

In [5]:
df[df['해제사유발생일'] != 'nan']

,단지명,계약년월,계약일,층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지,구,동,평,평형,계약요일,계약년,계약월,거래금액(억원)
96,그랑디오스,2022-01-01,8,5,2004.0,봉은사로51길 18,20220111.0,직거래,-,강남구,논현동,43.88,40평대,토요일,2022,1,15.914
100,논현아이파크,2022-02-01,23,7,2020.0,언주로 615,20220320.0,중개거래,서울 동대문구,강남구,논현동,15.66,10평대,수요일,2022,2,10.000
116,두영,2021-11-01,2,4,2004.0,논현로124길 8,20211231.0,직거래,-,강남구,논현동,28.00,20평대,화요일,2021,11,7.300
136,마일스디오빌,2022-08-01,12,15,2004.0,학동로 165,20220818.0,중개거래,서울 강남구,강남구,논현동,11.98,10평대,금요일,2022,8,3.850
140,신동아(22),2021-09-01,15,6,1997.0,강남대로146길 28,20211007.0,-,-,강남구,논현동,27.89,20평대,수요일,2021,9,17.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16840,동성3,2021-09-01,11,13,1997.0,신내로 110,20211111.0,-,-,중랑구,신내동,44.51,40평대,토요일,2021,9,9.450
16846,동성3,2021-11-01,1,21,1997.0,신내로 110,20220125.0,중개거래,"경기 용인수지구, 서울 중랑구",중랑구,신내동,42.70,40평대,월요일,2021,11,9.350
16899,신내6대주,2021-09-01,1,12,1996.0,신내로19길 42,20211206.0,-,-,중랑구,신내동,19.72,10평대,수요일,2021,9,7.500
16946,중앙하이츠,2021-10-01,25,13,1996.0,봉화산로56길 123,20211228.0,-,-,중랑구,신내동,28.04,20평대,월요일,2021,10,7.450


In [6]:
# 시각화 및 통계
# 1. 계약년월별 구별 거래건수, 평균거래가격
df.pivot_table(index='계약년월',values='구',aggfunc=['count','mean'])

C:\Users\admin\AppData\Local\Temp\ipykernel_9532\3603924737.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  df.pivot_table(index='계약년월',values='구',aggfunc=['count','mean'])


,count
,구
계약년월,
2021-09-01,2802
2021-10-01,2291
2021-11-01,1443
2021-12-01,1174
2022-01-01,1130
2022-02-01,849
2022-03-01,1491
2022-04-01,1823


In [7]:
# 시각화 및 통계
# 2. 계약년월별 구별 거래취소건수


In [8]:
# 시각화 및 통계
# 3. 구별 거래취소건수


In [9]:
# 시각화 및 통계
# 4. 평형 별 거래금액, 평균거래가격


In [10]:
# 시각화 및 통계

# 5. 구별 거래취소건수

- EDA실시(통계, 시각화)
- 실거래가 회귀분석